In [6]:
import cv2
import numpy as np
import imutils
ARUCO_DICT = {
"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

In [7]:
"""
Method to incapsulate the imshow, just for prettify the code
"""
def imshow(img, name = "img"):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

"""
This method is the preprocessing pipeline. I needed to 
change it from the classic method because the 
first implementation wasn't enough efficient on 
extracting the corners without introducing noise inside
its output.
"""
def bilateralFilteringPreprocessing(img):
    
    #Get only one channel
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #Blur the image using gaussian blur
    imgBlur = cv2.GaussianBlur(gray,(7,7),2)
    
    # Smoothing without removing edges.
    gray_filtered = cv2.bilateralFilter(gray, 9, 60, 120)
  
    # Applying the canny filter
    edges_filtered = cv2.Canny(gray_filtered, 30, 50) 

    return edges_filtered

path = "C:/Users/User Pc/Desktop/Progetto OpenCV/All notebooks/prototype_aruco_v_0.4_real.jpg"
img = cv2.imread(path)
imshow(bilateralFilteringPreprocessing(img))

In [8]:
"""
Object that contains the parameters extracted from cv2.aruco.DetectMarkers
just for cleaner code
"""
class arucoBox():
    def __init__(self,topLeft, topRight, bottomRight, bottomLeft, idx):
        self.topLeft = topLeft
        self.topRight = topRight
        self.bottomRight = bottomRight
        self.bottomLeft = bottomLeft
        self.idx = idx

In [9]:
"""
Given an image in input, returns a list that contains
the arucoBox object identified from cv2.aruco.detectMarkers
"""
def getArucos(img):
    
    #Get the arucoDict defined earlier
    arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT["DICT_4X4_50"])
    
    #Generate the detector obj
    arucoParams = cv2.aruco.DetectorParameters_create()
   
    #Detect the different arucoBoxes inside the img
    (corners, ids, rejected) = cv2.aruco.detectMarkers(img, arucoDict, parameters=arucoParams)
    arucos = []
    for (i,value) in zip(ids,corners):
        
        #Unpack the aruco coordinates
        (topLeft, topRight, bottomRight, bottomLeft) = value[0] 
        
        #Store them inside an array
        arucos.append(arucoBox(topLeft, topRight, bottomRight, bottomLeft,i[0]))
        
    return arucos

arucos = getArucos(img)

In [10]:
"""
Given a list of Arucos Markers and the image, gets 
the references using the markers with idx 1 and 2.
Also is useful to return the range of the xAxis so that
we can reuse it later to cut the chemical pads from the photo
"""
def getReferences(arucos,img):
    #For the way the tester its structured, we need just the idx1 and 2
    aruco_1 = [aruco for aruco in arucos if aruco.idx == 1][0]
    aruco_2 = [aruco for aruco in arucos if aruco.idx == 2][0]
    return img[int(aruco_2.bottomRight[1]):int(aruco_1.topLeft[1]),
              int( aruco_2.bottomRight[0]):int(aruco_1.topLeft[0])],int( aruco_2.bottomRight[0]),int(aruco_1.topLeft[0])

references,x_right,x_left = getReferences(arucos,img)
imshow(references)

In [11]:
"""
Method that return all the lines in the tester and also the min and max points 
from them, so that we can crop the image
"""
def getContourWithDefinedNumber(img, numberDesired):
    
  imgContour = img.copy()
  imgCanny = bilateralFilteringPreprocessing(img)
  #Store couple [[x_Start,y_Start], [X_End,y_End]] for each contour that we found
  #with the desired number of boundary points
  output = []

  #Extract all the contours from the img
  contours,hierarchy = cv2.findContours(imgCanny,cv2.RETR_LIST,
                                        cv2.CHAIN_APPROX_NONE)
  #For each contour found
  for cnt in contours:
        
    #Calculate the approximative Polygon to extract only the lines from the photo
    peri = cv2.arcLength(cnt,True)
     
    #Find the approximative Polygon based on the perimeter and the contour
    #approx is a list of points that define the polygon
    approx = cv2.approxPolyDP(cnt,0.02 * peri,True)
    
    #Number of corners of the polygon
    objCor = len(approx)
    
    #Return only the contours that have only the desired value
    if (objCor == numberDesired):
      
      #Draw the desired contour
      cv2.drawContours(imgContour,cnt,-1,(255,0,0),1)
      
      #Define the boundingRect so that, in the case of a line o a square, we have
      #The left coordinate, the width and the height
      x,y,w,h = cv2.boundingRect(approx)
      
      #Just for us to visualize the output
      cv2.rectangle(imgContour,(x,y),(x+w,y+h),(0,255,0),1)
        
      output.append(approx)
  #Show the output
  imshow(imgContour,str(numberDesired))

#Extract the lines from the tester using Canny Algorithm and cv2.drawContours
getContourWithDefinedNumber(img,2)

In [12]:
"""
Object that packs different methods useful for the color extraction from 
each bounding box
"""
class boxObject():
    
    #Init method
    def __init__(self,x,y,w,h):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.area = w*h
    
    #Given an image and a radius(predefine at 0.3), the method return the RGB of the area defined by the radius.
    def colorInside(self,img, radius = 0.1):
        
        #Extract the bounding Rect from the photo
        imageInside = img[self.y:self.y+self.h,self.x:self.x+self.w].copy()
        
        #Define the radius based on the w and h of the box
        radiusX = round(radius * self.w) 
        radiusY = round(radius * self.h)
        
        #Find the center from radiusX and radiusY
        center = imageInside[radiusX : 2*radiusX, radiusY : 2*radiusY]
        
        #Return the BGR of the area
        return [round(np.mean(center[:,:,0])),round(np.mean(center[:,:,1])),round(np.mean(center[:,:,2]))]
    
    def print(self):
        print(self.x,self.y,self.h,self.w)
    
    #Check if other is a duplicate of self
    def is_duplicate_of(self, other):
        x_dist = abs(self.x - other.x)
        y_dist = abs(self.y - other.y)
        area_div = self.area/other.area
        return (x_dist < 10 and 
                y_dist < 10 and 
                area_div > 0.9)

In [13]:
"""
Given an image and the number of vertex desired, returns a list
that contains the (x,y,w,h) of each elements that has pol edges
Used to extract the coordinates of each square
"""
def getPolygonCoords(img,pol = 2):
  imgContour = img.copy()
  #Preprocessing the image
  imgCanny = bilateralFilteringPreprocessing(img)
    
  #Find the contours so that we can choose the ones desired
  contours,hierarchy = cv2.findContours(imgCanny,cv2.RETR_LIST,
                                        cv2.CHAIN_APPROX_NONE)
  #Output array
  boxes = []
    
  #For each contours
  for cnt in contours:
    
    #Find the perimeter of the contour
    peri = cv2.arcLength(cnt,True)
    
    #Find the approximative Polygon based on the perimeter and the contour
    #approx is a list of points that define the polygon
    approx = cv2.approxPolyDP(cnt,0.02 * peri,True)
    
    #If has the desired number of corners
    if (len(approx) == pol):
      
      #Draw for each contour it's shape
      cv2.drawContours(imgContour,cnt,-1,(0,255,0),1)
    
      #Find the bounding rect of the approximative polygon
      x,y,w,h = cv2.boundingRect(approx)
        
      #Append it to the output array
      boxes.append(boxObject(x,y,w,h))
  
  #Just to show the different boxes extracted
  imshow(imgContour)
  return boxes

"""
Given an image in input, returns the (min,max) of the y_coords of the lines inside the photo.
Useful to isolate the references\chemical_pads
"""
def getLinesCoords(img):
    l = getPolygonCoords(img,2)
    #Choose only the horizontal lines inside the img
    lines = [value for value in l if(value.h/value.w <0.1)]
    #Compact way to iterate,using the generators functions
    y_max = max(value.y for value in lines)
    y_min = min(value.y for value in lines)
    return y_min,y_max

#We just need the min or the max because the image contains for now
#Just one horizontal line
y_chemical_line,_ = getLinesCoords(img)

chemical_pads = img[y_chemical_line:,x_right:x_left]

#The image of the chemical pads extracted from the tester
imshow(chemical_pads)
#The contours extracted from the chemical pads image
getContourWithDefinedNumber(chemical_pads,4)